# Calculating UHE neutrino-nucleon cross sections and uncertainties using QCD cross section calculations and parton distribution functions

*Author*            - Pawel Maciej Plesniak

*Student number*    - 19134004

*Course*            - MSc Scientific Computing

*Supervisor*        - Professor R. S. Thorne

*Second Supervisor* - Professor D. S. Waters

Before evaluating the plotting functions themselves, it is noted that the first cell contains all the internal definitions associated with the uncertainty calculation methods. This only needs to be evaluated once, as it changes the directory from cwd to "Data Sets".

In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os


def read_data(int_type):
    """
################################################################################
    This function reads the CSV file associated with the int_type, which is a
    logarithmic data set, and converts it into a numpy array in non-logarithmic
    form. The units of the non-logarithmic cross sections are cm^{-2}, and span
    the full range of 51 eigenvector PDFs. The cross sections are all indexed
    by [PDF][Energy], where the central cross section is in the first row. The
    exceptions to this are the 'toyLO' and 'toyNLO' data sets, which contain
    the non-logarithmic structure functions at corresponding orders. These
    structure functions are modelled based on a toy set of PDFs for the LHC.
    The variable is:
        1. int_type - this string is the name of the CSV file containing the
                      cross sections of the named interaction type. The CSV
                      files are all named by the corresponding interaction
                      types.
            *nc - neutral current neutrino hadron DIS cross sections in the
                  full energy range [10^2, 10^12] GeV with step 10^0.2 GeV.
            *cc - charged current neutrino hadron DIS cross sections in the
                  full energy range [10^2, 10^12] GeV with step 10^0.2 GeV.
            *ccS - charged current neutrino hadron DIS cross sections in the
                   short energy range [10^2, 10^4] GeV with step 10^0.05 GeV.
            *ncA - neutral current antineutrino hadron DIS cross sections in
                   the full energy range [10^2, 10^12] GeV with step 10^0.2
                   GeV.
            *ccA - charged current antineutrino hadron DIS cross sections in
                   the full energy range [10^2, 10^12] GeV with step 10^0.2
                   GeV.
            *ccAS - charged current antineutrino hadron DIS cross sections in
                    the short energy range [10^2, 10^4] GeV with step 10^0.05
                    GeV.
            *ncY - neutral current neutrino hadron differential DIS cross
                   sections in inelasticity with energy in the full range
                   [10^7, 10^12] GeV with step 10^0.1 GeV and inelasticity in
                   the full range [10^-5, 0] with step 10^0.1.
            *ccY - charged current neutrino hadron differential DIS cross
                   sections in inelasticity with energy in the full range
                   [10^7, 10^12] GeV with step 10^0.1 GeV and inelasticity in
                   the full range [10^-5, 0] with step 10^0.1.
            *ncAY - neutral current antineutrino hadron differential DIS cross
                   sections in inelasticity with energy in the full range
                   [10^7, 10^12] GeV with step 10^0.1 GeV and inelasticity in
                   the full range [10^-5, 0] with step 10^0.1.
            *ccAY - charged current antineutrino hadron differential DIS cross
                   sections in inelasticity with energy in the full range
                   [10^7, 10^12] GeV with step 10^0.1 GeV and inelasticity in
                   the full range [10^-5, 0] with step 10^0.1.
            *charm - charged current neutrino hadron to charmed hadron DIS
                     cross sections in the short energy range [10^2, 10^4] GeV
                    with step 10^0.05 GeV.
            *charmA - charged current antineutrino hadron to charmed hadron DIS
                      cross sections in the short energy range [10^2, 10^4] GeV
                      with step 10^0.05 GeV.
            *toyLO - structure function F_2, F_L and xF_3 data for leading
                     order charged current interactions, indexed by neutrino
                     type (23 for neutrinos, 24 for antineutrinos), x, Q^2,
                     F_2, F_L, and xF_3.
            *toyNLO - structure function F_2, F_L and xF_3 data for next to
                      leading order charged current interactions, indexed by
                      neutrino type (23 for neutrinos, 24 for antineutrinos),
                      x, Q^2, F_2, F_L, and xF_3.
            *ccLO - charged current neutrino hadron DIS leading order
                     cross sections in the full energy range [10^2, 10^12] GeV
                     with step 10^0.2 GeV.
            *ccNLO - charged current antineutrino hadron DIS next to leading
                      order cross sections in the full energy range [10^2,
                      10^12] GeV with step 10^0.2 GeV.
################################################################################
    """
    if int_type not in ['nc', 'cc', 'ccS', 'ncA', 'ccA', 'ccAS', 'ncY', 'ccY',
                        'ncAY', 'ccAY', 'charm', 'charmA', 'toyLO', 'toyNLO',
                        'ccLO', 'ccNLO']:
        raise ValueError('This is not a valid type of interaction type - '
                         'please ensure that the passed int_type is selected '
                         'from the following: "nc", "cc", "ccS", "ncA", '
                         '"ccA", "ccAS", "ncY", "ccY", "ncAY", "ccAY", '
                         '"charm", "charmA", "toyLO", "toyNLO", "ccLO" and '
                         '"ccNLO". See the function documentation for more '
                         'information on each data set.')
    with open(int_type + '.csv', newline='') as csvfile:
        logarithmic_data = list(csv.reader(csvfile))
    if int_type[-1] == 'Y':
        converted_data =\
            np.array([np.array([np.array([
                float(y_index.replace('{', '').replace('}', ''))
                for y_index in E_index[1:-2].split(', ')])
                                for E_index in pdf_index])
                      for pdf_index in logarithmic_data])
        return(10 ** converted_data)
    else:
        converted_data = np.array([np.array([float(E_index) for E_index in
                                             pdf_index]) for pdf_index
                                   in logarithmic_data])
        if int_type[:3] == 'toy':
            return(converted_data)
        else:
            return(10 ** converted_data)


def _data_check(data, expected_shape_len):
    """
################################################################################
    *** Intended for internal use only***
    This function provides checks on the data object for its type and length.
    The variables are:
        1. data - this numpy array is the set which is being checked.
        2. expected_shape_len - this int determines whether the data is
                                expected to be two or three dimensional.
################################################################################
    """
    if type(data) != np.ndarray:
        raise ValueError('This is not a valid data type - please ensure that '
                         'the data is of type numpy.ndarray.')
    if expected_shape_len not in [2, 3]:
        raise ValueError('This is not a valid expected_shape_length - please '
                         'ensure that this is either 2 (for working with '
                         'cross section or structure function data) or 3 (for '
                         'working with inelasticity differential cross '
                         'section data).')
    if len(data.shape) != expected_shape_len:
        if expected_shape_len == 2:
            raise ValueError('This is not a valid data shape - please ensure '
                             'that your data is a two dimensional numpy '
                             'array.')
        elif expected_shape_len == 3:
            raise ValueError('This is not a valid data shape - please ensure '
                             'that your data is a three dimensional numpy '
                             'array.')
    return


def _uncert_indexes(data, expected_shape_len):
    """
################################################################################
    *** Intended for internal use only. ***
    This function detemines the energy and pdf reference indexes used for the
    calculation of the uncertainties. This is returned as a list of the pdf
    index and the energy index.
    The variables are:
        1. data - this 2 or 3 dimensional numpy array contains the cross
                  sections indexed by their eigenvector PDF and energy (and
                  inelasticity).
        2. expected_shape_len - this int shows the expected dimension of the
                                presented data:
            * 2 - this is expected when working with cross section data.
            * 3 - this is expected when working with inelasticity differential
                  cross section data.
################################################################################
    """
    _data_check(data, expected_shape_len)
    pdf_index = np.arange(1, len(data)//2 + 1)
    E_index = np.arange(len(data[0]))
    return(pdf_index, E_index)


def _format_uncertainty(uncertainty_data, expected_shape_len):
    """
################################################################################
    *** Intended for internal use only. ***
    This function duplicates the uncertainty cross section results. As there
    are only 25 eigenvector PDFs, but 50 directions, for each direction one
    requires both directions to have the same positive and negative error, thus
    returns the data duplicated in the order presented. The central set also
    has to be accounted for, thus an array of zeros is appended at the
    beginning of each uncertainty numpy array, as this is where the central
    cross section is.
    The variables are:
        1. uncertainty_data - this 2 dimensional numpy array contains the
                              required uncertainties that require duplication.
        2. expected_shape_len - this int distinguishes the two dimensional data
                               cases (cross sections and structure functions)
                               from the three dimensional ones (inelasticity
                               differential cross sections).
################################################################################
    """
    _data_check(uncertainty_data, expected_shape_len)
    if expected_shape_len == 2:
        return(np.append(np.zeros((1, uncertainty_data.shape[1])),
                         np.repeat(uncertainty_data, 2, axis=0), axis=0))
    elif expected_shape_len == 3:
        return(np.append(np.array([np.repeat(
                                   np.zeros((1, uncertainty_data.shape[1])),
                                   uncertainty_data.shape[1], axis=0)]),
                         np.repeat(uncertainty_data, 2, axis=0), axis=0))
    return


def _cs_uncertainty(data, method='a', _fraction_flag=False, _sum_flag=False):
    """
################################################################################
    *** Intended for internal use only. ***
    This function calculates the uncertainties associated with the cross
    sections and returns a two dimensional numpy array of errors indexed by the
    eigenvector PDF and the energy (if _sum_flag is False), or a one
    dimensional array of uncertainties indexed by the energy (if _sum_flag is
    True).
    The variables are:
        1. data - this 2 dimensional numpy array contains the cross sections,
                  indexed by their eigenvector PDF and energy.
        2. method - this string determines the method in which the
                    uncertainties are calculated.
            * 'p' - this method returns the errors in the positive direction.
            * 'n' - this method returns the errors in the negative direction.
            * 'a' - this method returns the errors averaged between the postive
                    and negative directions.
        3. _fraction_flag - this flag determines whether the cross sections
                            should be divided by the corresponding eigenvector
                            PDF cross sections (True) or not (False).
        4. _sum_flag - this flag determines whether the quadratic summation
                       should be applied (True) or not (False).
################################################################################
    """
    expected_shape_len = 2
    _data_check(data, expected_shape_len)
    if method not in ['p', 'm', 'a']:
        raise ValueError('This is not a valid method - please ensure that the '
                         'passed method is either "p", "m", or "a". See the '
                         'function documentation for more information on the '
                         'various methods.')
    if type(_fraction_flag) != bool:
        raise ValueError('This is not a valid _fraction_flag type - please '
                         'ensure that the passed _fraction_flag is a boolean.')
    if type(_sum_flag) != bool:
        raise ValueError('This is not a valid _sum_flag type - please ensure '
                         'that the passed _fraction_flag is a boolean.')

    pdf_index, E_index = _uncert_indexes(data, expected_shape_len)

    if method == 'p':
        delta = np.array([np.array([max(data[2 * i - 1][E] - data[0][E],
                                        data[2 * i][E] - data[0][E], 0)
                                    for E in E_index]) for i in pdf_index])
    elif method == 'm':
        delta = np.array([np.array([max(data[0][E] - data[2 * i - 1][E],
                                        data[0][E] - data[2 * i][E], 0)
                                    for E in E_index]) for i in pdf_index])
    elif method == 'a':
        return(0.5*(_cs_uncertainty(data, 'p', _fraction_flag=_fraction_flag,
                                    _sum_flag=_sum_flag) +
                    _cs_uncertainty(data, 'm', _fraction_flag=_fraction_flag,
                                    _sum_flag=_sum_flag)))

    if _fraction_flag:
        delta = _format_uncertainty(delta, expected_shape_len)/data
    else:
        delta = _format_uncertainty(delta, expected_shape_len)

    if _sum_flag:
        return(np.sqrt(np.sum(delta**2, axis=0)))
    else:
        return(delta)


def _cs_y_uncertainty(data, method='a', _fraction_flag=False, _sum_flag=False):
    """
################################################################################
    *** Intended for internal use only. ***
    This function calculates the uncertainties associated with the inelasticity
    distributed cross sections using the eigenvector PDF values. The variables
    are as per '_cs_uncertainty'. Here, _sum_flag is set to False to enable the
    _construct_plots function to work smoothly.
################################################################################
    """
    expected_shape_len = 3
    pdf_index, E_index = _uncert_indexes(data, 3)
    if method not in ['p', 'm', 'a']:
        raise ValueError('This is an invalid method - please ensure that the '
                         'passed method is either "p", "m", or "a". See the '
                         'function documentation for more information on the '
                         'various methods.')
    y_index = np.arange(len(data[0][0]))
    if method == 'p':
        delta = np.sqrt(np.sum((np.array([np.array(
            [np.array([max(data[2 * i - 1][E][y] - data[0][E][y],
                           data[2 * i][E][y] - data[0][E][y], 0)
                       for y in y_index])
             for E in E_index])for i in pdf_index]))**2, axis=0))
    elif method == 'm':
        delta = np.sqrt(np.sum((np.array([np.array([np.array(
            [max(data[0][E][y] - data[2 * i - 1][E][y],
                 data[0][E][y] - data[2 * i][E][y], 0) for y in y_index])
                                                    for E in E_index])
                                          for i in pdf_index]))**2, axis=0))
    elif method == 'a':
        return(
            0.5*(_cs_y_uncertainty(data, 'p', _fraction_flag=_fraction_flag) +
                 _cs_y_uncertainty(data, 'm', _fraction_flag=_fraction_flag)))
    if _fraction_flag:
        return(delta/data[0])
    else:
        return(delta)


def _plot_check(uncert_plot_method, uncert_plot_style, plot_inline, log_flag,
                plot_uncertainties):
    """
################################################################################
    *** Intended for internal use only. ***
    This function checks whether the passed plotting parameters for the 3d plot
    functions plot_3d_cross_section and plot_3d_cross_section_y are
    appropriate.
    The variables are:
        1. uncert_plot_method - this string represents the method in which the
                                uncertainties are calculated.
        2. uncert_plot_style - this string represents the method in which the
                               uncertainties are calculated in the plots.
        3. plot_inline - this boolean determines whether the plot should be
                         inline in the output console, or should be opened
                         in a new window.
        4. log_flag - this boolean determines whether the data should have its
                      base 10 logarithm taken (True) or not (False).
        5. plot_uncertainties - this boolean determines whether or not to plot
                                the uncertainties.
################################################################################
    """
    if uncert_plot_method not in ['averaged', 'positive', 'negative',
                                  'asymmetric']:
        raise ValueError('This is not a valid type of uncertainty plot - '
                         'please select either "averaged", "positive", '
                         '"negative" or "asymmetric".')
    if uncert_plot_style not in ['relative', 'absolute', 'fractional']:
        raise ValueError('This is not a valid style of uncertainty plot - '
                         'please select either "relative", "absolute" or '
                         '"fractional".')
    a = [type(plot_inline) == bool, type(log_flag) == bool,
         type(plot_uncertainties) == bool]
    if not all(a):
        raise ValueError('This is not a valid type of flag - please ensure '
                         'that plot_inline, log_flag and plot_uncertainties '
                         'are all boolean.')
    test = [not plot_uncertainties and uncert_plot_method != 'asymmetric',
            not plot_uncertainties and uncert_plot_style != 'relative']
    if any(test):
        raise ValueError("Warning: please ensure that plot_uncertainties "
                         "is True if the the desired uncertainties are to "
                         "be plotted.")
    return


def _add_plot(data, plot_data, log_flag, plot_label, plot_labels):
    """
################################################################################
    *** Intended for internal use only. ***
    This function appends the data sets of interest to the corresponding
    arrays. It returns a list of data plot_data with corresponding labels
    plot_labels to be plotted.
    The variables are:
        1. data - this numpy array contains the data set that is being plotted.
        2. plot_data - this list contains the data arrays to be plotted.
        3. log_flag - this flag determines whether the data array should have
                      its base 10 logarithm taken (True) or not (False).
        4. plot_label - this string is the label for the corresponding data for
                        use in the legend.
        5. plot_labels - this list contains all the strings which correspond to
                         the arrays in plot_data which are to be plotted.
################################################################################
    """
    if type(data) != np.ndarray:
        raise ValueError('This is not a valid type of data - please ensure '
                         'that the passed data item is a numpy array.')
    if type(plot_data) != list:
        raise ValueError('This is not a valid type of plot_data - please '
                         'ensure that the passed plot_data item is a list.')
    if type(log_flag) != bool:
        raise ValueError('This is not a valid type of log_flag - please '
                         'ensure that the passed log_flag item is a boolean.')
    if type(plot_label) != str:
        raise ValueError('This is not a valid type of plot_label - please '
                         'ensure that the passed plot_label is a string.')
    if type(plot_labels) != list:
        raise ValueError('This is not a valid type of plot_labels - please '
                         'ensure that the passed plot_labels is a list.')

    plot_labels.append(plot_label)
    if log_flag:
        plot_data.append(np.log10(data))
    else:
        plot_data.append(data)
    return([plot_labels, plot_data])


def _labels(int_type, log_flag, uncert_plot_style):
    """
################################################################################
    *** Intended for internal use only. ***
    This function returns the appropriate z_label and title_label for the
    relevant plots.
    The variables are:
        1. data - this string determines the labels to be returned.
        2. log_flag - this flag determines whether title_label should be
                      capitalized (True) or not (False).
        3. uncert_plot_style - this string determines how z_label should be
                               amended.
################################################################################
    """
    int_types = ['cc', 'ccA', 'ccS', 'ccAS', 'nc', 'ncA', 'charm', 'charmA',
                 'ccNLO', 'ccANLO', 'ccY', 'ccAY', 'ncY', 'ncAY', 'ccLO',
                 'ccNLO']
    z_labels = [r'$\mathregular{\sigma^{CC}}(\nu_\ell\, H \rightarrow'
                r'\ell^\mp\, X)$',
                r'$\mathregular{\sigma^{CC}}(\bar{\nu}_\ell\, H '
                r'\rightarrow \ell^\mp\, X)$',
                r'$\mathregular{\sigma^{CC}}(\nu_\ell\, H \rightarrow'
                r'\ell^\mp\, X)$',
                r'$\mathregular{\sigma^{CC}}(\bar{\nu}_\ell\, H'
                r'\rightarrow \ell^\mp\, X)$',
                r'$\mathregular{\sigma^{NC}}(\nu_\ell\, H\rightarrow'
                r'\nu_\ell\, X)$',
                r'$\mathregular{\sigma^{NC}}(\bar{\nu}_\ell\, H'
                r'\rightarrow \bar{\nu}_\ell\, X)$',
                r'$\mathregular{\sigma^{CC}}(\nu_\ell\, H '
                r'\rightarrow\nu_\ell\, c\, X)$',
                r'$\mathregular{\sigma^{CC}}(\bar{\nu}_\ell\, H'
                r'\rightarrow \bar{\nu}_\ell\, c\, X)$',
                r'$\mathregular{\sigma^{CC}_{NLO}}(\nu_\ell\, H '
                r'\rightarrow\nu_\ell\, c\, X)$',
                r'$\mathregular{\sigma^{CC}_{NLO}}(\bar{\nu}_\ell\, H'
                r'\rightarrow \bar{\nu}_\ell\, c\, X)$',
                r'$\frac{d\sigma^{CC}}{dy}'
                r'(\nu_\ell\, H \rightarrow\ell^\mp\, X)$',
                r'$\frac{d\sigma^{CC}}{dy}'
                r'(\bar{\nu}_\ell\, H \rightarrow \ell^\mp\, X)$',
                r'$\frac{d\sigma^{NC}}{dy}'
                r'(\nu_\ell\, H \rightarrow \nu_\ell\, X)$',
                r'$\frac{d\sigma^{NC}}{dy}'
                r'(\bar{\nu}_\ell\,H\rightarrow\bar{\nu}_\ell\,X)$',
                r'$\mathregular{\sigma^{CC}}(\nu_\ell\, H \rightarrow'
                r'\ell^\mp\, X)$',
                r'$\mathregular{\sigma^{CC}_{NLO}}(\nu_\ell\, H'
                r'\rightarrow \ell^\mp\, X)$']
    title_labels = ['charged current neutrino',
                    'charged current antineutrino',
                    'charged current neutrino',
                    'charged current antineutrino',
                    'neutral current neutrino',
                    'neutral current antineutrino',
                    'charged current neutrino to charmed hadron',
                    'charged current antineutrino to charmed hadron',
                    'charged current neutrino NLO',
                    'charged current antineutrino NLO',
                    'charged current neutrino',
                    'charged current antineutrino',
                    'neutral current neutrino',
                    'neutral current antineutrino',
                    'leading order charged current neutrino',
                    'next-to leading order charged current neutrino']

    int_type_index = int_types.index(int_type)
    title_label = title_labels[int_type_index]
    z_label = z_labels[int_type_index]

    if not log_flag:
        title_label = title_label.capitalize()

    if uncert_plot_style == 'fractional':
        int_type = int_type[:2].upper()
        if int_type[-3:] == 'NLO':
            z_label.replace(r"sigma^{" + int_type + "}_{NLO}", r"$frac{\sigma"
                            + r"^{" + int_type + r"_{NLO}}}{\sigma_0}$")
        elif int_type[-1] == 'Y':
            z_label.replace("frac", r"$frac{1}{\sigma^{" + int_type + "}_0}"
                            + r"\frac$")
        else:
            z_label.replace("sigma^{" + int_type + "}", r"$frac{\sigma^{$"
                            + int_type + r"$}}{\sigma_0}$")

    elif uncert_plot_style == 'absolute':
        z_label.replace("sigma", r"$Delta\sigma$")

    return(z_label, title_label)


def _construct_plots(int_type, log_flag, plot_uncertainties,
                     uncert_plot_method, uncert_plot_style, _sum_flag,
                     uncert_function):
    """
################################################################################
    *** Intended for internal use only. ***
    This function returns the appropriate lists of data, labels, and colors
    for the plotting functions.
    The variables are:
        1. data - this two or three dimensional numpy array contains the data
                  that is to be analysed.
        2. log_flag - this flag determines whether title_label should be
                      capitalized (True) or not (False), as well as whether the
                      data should have its base 10 logarithm taken.
        3. uncert_plot_method - this string determines how the uncertainties
                                are calculated.
        3. uncert_plot_style - this string determines how the uncertainties are
                               presented in the plots.
        4. _sum_flag - this flag determines whether the quadratic sum should be
                       applied to the data over the PDFs.
        5. uncert_function - this function handle determines the method in
                             which the data uncertainties are calculated. This
                             should be either '_cs_uncertainty' or
                             '_cs_y_uncertainty'.
    This function returns:
        1. plot_data - this list contains the data sets that are to be plotted.
        2. plot_labels - this list contains the corresponding labels to the
                         data being plotted.
        3. colors - this list contains the colors of the plots. The objective
                    is to have all central plots in black, all positive and
                    negative uncertainties in blue and red respectively.
        4. log_title - this string will be the used in the title to show that
                       the data being plotted is logarithmic.
        5. z_label_end - this string will be not empty if the uncert_plot_style
                         is not fractional, and will contain the units of the
                         cross sections being calculated, if the fractional
                         plot is not being shown.
################################################################################
    """
    data = read_data(int_type)
    d_len = len(data.shape)
    plot_data = []
    plot_labels = []
    plot_colors = []
    z_label_end = r'$\mathregular{cm^{-2}}$'
    if log_flag:
        log_title = r'$\mathregular{Log_{10}}$ '
    else:
        log_title = ''

    test = [not plot_uncertainties, plot_uncertainties and uncert_plot_style ==
            'relative']
    if any(test):
        if int_type[:4] != 'charm':
            if d_len == 2:
                central_label = r'$\mathregular{\sigma^{'\
                                + int_type[:2].upper() + r'}}$'
            elif d_len == 3:
                central_label = r'$\mathregular{\frac{\sigma^{' \
                                + int_type[:2].upper() + r'}}{\sigma_0}}$'
        else:
            central_label = r'$\mathregular{\sigma^{CC}_{charm}}$'
        if d_len == 2:
            central_pdf = data
        else:
            central_pdf = data[0]
        plot_labels, plot_data = _add_plot(central_pdf, plot_data, log_flag,
                                           central_label, plot_labels)
        plot_colors.append('black')

    if plot_uncertainties:
        if log_flag and uncert_plot_style == 'fractional':
            raise ValueError("Warning: The full range of values may not "
                             "display as many of the cross section "
                             "uncertainties from individual PDFs are null, "
                             "thus by setting log_flag to true this raises "
                             "library runtime and divergence errors.")
        if uncert_plot_style == 'fractional':
            frac_flag = True
            z_label_end = ''
        elif uncert_plot_style in ['relative', 'absolute']:
            frac_flag = False

        if uncert_plot_method == 'averaged':
            delta = [uncert_function(data, 'a', _fraction_flag=frac_flag,
                                     _sum_flag=_sum_flag), 'a']
        elif uncert_plot_method == 'asymmetric':
            delta = [uncert_function(data, 'p', _fraction_flag=frac_flag,
                                     _sum_flag=_sum_flag),
                     uncert_function(data, 'm', _fraction_flag=frac_flag,
                                     _sum_flag=_sum_flag)]
        elif uncert_plot_method == 'positive':
            delta = [uncert_function(data, 'p', _fraction_flag=frac_flag,
                                     _sum_flag=_sum_flag), '+']
        elif uncert_plot_method == 'negative':
            delta = [uncert_function(data, 'm', _fraction_flag=frac_flag,
                                     _sum_flag=_sum_flag), '-']

        if len(data.shape) == 3:
            data = data[0]

        if type(delta[1]) == str:
            if uncert_plot_style != 'fractional':
                if delta[1] == '+':
                    plot_labels, plot_data = _add_plot(data + delta[0],
                                                       plot_data, log_flag,
                                                       r'$\Delta\sigma^+$',
                                                       plot_labels)
                elif delta[1] == '-':
                    plot_labels, plot_data = _add_plot(data - delta[0],
                                                       plot_data, log_flag,
                                                       r'$\Delta\sigma^-$',
                                                       plot_labels)
                elif delta[1] == 'a':
                    plot_labels, plot_data = _add_plot(data + delta[0],
                                                       plot_data, log_flag,
                                                       r'$\Delta\sigma^+$',
                                                       plot_labels)
                    plot_labels, plot_data = _add_plot(data - delta[0],
                                                       plot_data, log_flag,
                                                       r'$\Delta\sigma^-$',
                                                       plot_labels)
            elif uncert_plot_style == 'fractional':
                if delta[1] == '+':
                    plot_labels, plot_data = _add_plot(delta[0], plot_data,
                                                       log_flag,
                                                       r'$\Delta\sigma^+$',
                                                       plot_labels)
                elif delta[1] == '-':
                    plot_labels, plot_data = _add_plot(-delta[0], plot_data,
                                                       log_flag,
                                                       r'$\Delta\sigma^-$',
                                                       plot_labels)
                elif delta[1] == 'a':
                    plot_labels, plot_data = _add_plot(delta[0], plot_data,
                                                       log_flag,
                                                       r'$\Delta\sigma^+$',
                                                       plot_labels)
                    plot_labels, plot_data = _add_plot(-delta[0], plot_data,
                                                       log_flag,
                                                       r'$\Delta\sigma^-$',
                                                       plot_labels)
        else:
            if uncert_plot_style != 'fractional':
                plot_labels, plot_data = _add_plot(data + delta[0],
                                                   plot_data, log_flag,
                                                   r'$\Delta\sigma^+$',
                                                   plot_labels)
                plot_labels, plot_data = _add_plot(data - delta[1],
                                                   plot_data, log_flag,
                                                   r'$\Delta\sigma^-$',
                                                   plot_labels)
            elif uncert_plot_style == 'fractional':
                plot_labels, plot_data = _add_plot(delta[0], plot_data,
                                                   log_flag,
                                                   r'$\Delta\sigma^+$',
                                                   plot_labels)
                plot_labels, plot_data = _add_plot(-delta[1], plot_data,
                                                   log_flag,
                                                   r'$\Delta\sigma^-$',
                                                   plot_labels)

    if uncert_plot_method == 'positive':
        plot_colors.append('blue')
    elif uncert_plot_method == 'negative':
        plot_colors.append('red')
    else:
        plot_colors.append('blue')
        plot_colors.append('red')

    return(plot_data, plot_labels, plot_colors, log_title, z_label_end)


os.chdir(os.getcwd() + "\\Data Sets")


#### This section defines the preliminary functions defined in the above script
#### The remaining function documentations are all presented with the function definitions
`read_data`

This function imports the csv associated with a particular interaction type `int_type`. The description of each of the interaction types is presented in the script and the report. This function operates on two different data set structures, thus the need for the "Y" if statement - the cross section data sets and the toy PDF structure function data sets are two dimensional CSVs, whereas the inelasticity differential data sets are three dimensional CSVs. The cross section data sets are indexed by the eigenvector PDF, then the neutrino energy, and the toy PDF structure functions are indexed by [neutrino type, x, Q^2, F_2, F_L, xF_3], then the result from the integration. The inelasticity differential data sets have a further index for the inelasticity, hence a different method is required to split these data sets. In the function, the difference also includes a splitting and replacing function for each string, which accounts for the variation in storage methods of the CSV files, as the extra index requires the data to be stored in lists separated by commas. The replace function removes the list brackets, and the split function breaks down the strings of lists into elements.

`_data_check`

The function checks `data` for being either a two or three dimensional numpy array, and `expected_shape_len` is also checked for appropriate values of either 2 or 3. This is to check that `data` has the appropriate dimensions.

`_uncert_indexes`

This function returns the indexes used to reference the associated PDFs and energies for a given data set. `pdf_index` is given from 1 and is of length of only half of the eigenvector PDF sets as they are constructed with uncertainties in two directions. The numpy arrays for the PDF index `pdf_index` and energy index `E_index` are constructed and returned as a two element list.

`_format_uncertainty`

This function duplicates the uncertainties corresponding to each eigenvector PDF set along the axis in which they are presented. This is required as each eigenvector PDF set has two directions, thus for the three dimensional plots the uncertainty must be added (subtracted) from the corresponding PDF cross section. An array of zeros is added to the beginning of the uncertainty arrays as the central cross section uncertainty is null by definition of the approach taken. In the two dimensional uncertainty data case (cross sections), one requires a one dimensional array of zeros to be appended as the uncertainty data is one dimensional (indexed only by the energy). In the case of three dimensional uncertainty data (inelasticity differential cross sections), one requires a two dimensional array of zeros as the uncertainty data is two dimensional (also indexed by the inelasticity).

`_cs_uncertainty`

This function computes the uncertainties associated with a cross section set. `_data_check` is initially run to check 
`data` type and length against what is expected, and `_fraction_flag` and `_sum_flag` are checked for being boolean types. The indexes are computed using the `_uncert_indexes` function. The function applies the methods to calculate the uncertainties corresponding to each eigenvector PDF according to equations 2.3.12 and 2.3.13 for the positive and negative directions respectively. If `method` is `'p'`, only the uncertainties in the positive directions are calculated, if `method` is `'m'` only the uncertainties in the negative direction are applied, and if `method` is `'a'`, both of the aforementioned methods are used, and the result is averaged. Before returning, the data formatted using `_format_uncertainty`, `_fraction_flag` is checked, and if it is `True` the uncertainties are divided by the respective cross sections. The quadratic sum in these equations is then applied when `_sum_flag` is `True`, returning a one dimensional numpy array containing the uncertainty indexed by the energy, otherwise when `_sum_flag` is `False`, the quadratic sum is not applied and a two dimensional uncertainty array is returned indexed by the PDF then the neutrino
energy.

`_cs_y_uncertainty`

This function computes the uncertainties associated with a inelasticity differential cross section set, and works exactly as _cs_uncertainty does. The difference between the two functions is due to the inherent nature of the data sets being worked on - the inelasticity differential cross section sets are three dimensional numpy arrays, thus require an additional index to calculate the uncertainties. the _sum_flag here is not used as the resulting data for the plots is required to be two dimensional, thus the indexes for these uncertainties are the neutrino energy and the inelasticity. This flag is passed, however, to ensure that the _construct_plots
function works smoothly. 

`_plot_check`

This function verifies that the passed parameters are all of the correct type. `uncert_plot_method` and `uncert_plot_style` are checked against a list of accepted values.

`_add_plot `

This function appends the data set intended to be plotted `data` with a corresponding label 'plot_label' to a list of data `plot_data` and labels `plot_labels`. The function initially checks the passed parameters all for types. If `log_flag` is True then this function also takes the base 10 logarithm of the data before appending it to `plot_data`.

`_labels`

This function returns the correct z axis label and title label. This is done by indexing `int_types` to find the correct index, and the correct values are selected from the list. In the cae that `log_flag` is `False`, `title_label` is capitalized, and dependent on the `uncert_plot_style`, the title labels are edited.

`_construct_plots`

This function returns the list of data that is to be plotted `plot_data`, a list of the corresponding labels `plot_labels`, a list of the colors which the plots are to be of `plot_colors`, a string which is used in the title if the base 10 logarithm is to be taken `log_title`, and the appropriate ending to the z label `z_label_end`. The function initially computes the length of the data to ensure that the correct sets of data are plotted, and empty lists for `plot_data`, `plot_labels`, and `colors` are instantiated. The correct `log_title` is also assigned. If `plot_uncertainties` is `False`, or if `plot_uncertainties` is `True` and the `uncert_plot_style` is 'relative', then the central data sets are added to `plot_labels` and `plot_data` using the `_add_plot` function. Otherwise, if `plot_uncertainties` is `True` and so is `log_flag` with `uncert_plot_style` as 'fractional', then an warning message is printed as the fractional values will not require the logarithm to be taken to display the intended data, as the logarithms of the fractions will be plotted instead. If `uncert_plot_style` is set to 'fractional', then `frac_flag` is set to `True` which will return the appropriate functions when using `uncert_function`. Otherwise, `frac_flag` is set to `False`. The uncertainties are calculated using `uncert_function`, and appended to `plot_labels` and `plot_data` using the `add_plot` method. The central PDF values are chosen for the inelasticity differential cross sections as there is no way to plot this over the neutrino energy and inelasticity, as well as the PDF index. The correct colors are further appended based on the choice of 'uncert_plot_method'. 

In [3]:
def LO_cross_section(int_type, plot_inline=True, log_flag=True,
                     plot_uncertainties=False, uncert_plot_method='asymmetric',
                     uncert_plot_style='relative'):
    """
################################################################################
    This function plots a cross section data set (and its associated
    uncertainties) in three dimensions as a function of the eigenvector PDF set
    and the energy.
    The variables are:
        1. int_type - this string defines the type of interaction that is
                      being plotted. This should be a choice of the permitted
                      values of the function _read_data aside from the
                      inelasticity differential cross section and structure
                      function data.
        2. plot_inline - this flag determines whether the plot should be
                         displayed in the console (True) or in a new window
                         (False).
        3. log_flag - this flag determines whether the plots are presented in
                      base 10 logarithmic form (True) or not (False).
        4. plot_uncertainties - this flag determines whether or not the
                                uncertainties are to be plotted (True) or not
                                (False).
        5. uncert_plot_method - this string determines the method in which the
                                uncertainties are calculated and plotted
                                relative to the data.
            * 'averaged' - this calculates the errors in both the positive
                           and negative directions, averages the result, and
                           plots both errors.
            * 'asymmetric' - this calculates the errors in both the positive
                             and negative directions, and plots both errors.
            * 'positive' - this calculates the errors in the positive direction
                           only, and plots the positive error only.
            * 'negative' - this calculates the errors in the positive direction
                           only, and plots the positive error only.
        6. uncert_plot_style - this determines the presentation of the plot in
                               the graph.
            * 'relative' - this plots the central cross section with the
                           associated errors.
            * 'absolute' - this produces a plot of only the positive and
                           negative errors. The central cross section is not
                           included in this plot.
            * 'fractional' - this produces a plot with the errors divided
                             by the corresponding cross sections.
    For the fractional plots, the label of sigma_0 corresponds to the
    corresponding central PDF cross section, which the calculated errors are
    divided by.
################################################################################
    """
    _plot_check(uncert_plot_method, uncert_plot_style, plot_inline, log_flag,
                plot_uncertainties)

    data = read_data(int_type)
    if int_type not in ['ccS', 'ccAS', 'charm', 'charmA', 'ccNLO', 'ccANLO',
                        'nc', 'cc', 'ncA', 'ccA', 'ccLO']:
        raise ValueError("This is not a valid type of interaction for this "
                         "function - please select either 'nc', 'cc', 'ncA', "
                         "'ccA', 'ccS', 'ccAS', 'charm', or 'charmA'. If you "
                         "would like to see the 3d distribution for "
                         "inelasticity data, please see the "
                         "'plot_3d_cross_section_y' function.")

    if plot_inline:
        %matplotlib inline
    else:
        %matplotlib qt

    plot_items = _construct_plots(int_type, log_flag, plot_uncertainties,
                                  uncert_plot_method, uncert_plot_style,
                                  False, _cs_uncertainty)
    plot_data, plot_labels = plot_items[0], plot_items[1]
    colors, log_title = plot_items[2], plot_items[3]
    z_label_end = plot_items[4]
    z_label, title_label = _labels(int_type, log_flag, uncert_plot_style)

    if int_type in ['ccS', 'ccAS', 'charm', 'charmA']:
        E_range = np.linspace(2, 4, num=len(data[0]))
    elif int_type in ['nc', 'cc', 'ncA', 'ccA', 'ccLO', 'ccNLO']:
        E_range = np.linspace(2, 12, num=len(data[0]))

    pdf_range = np.arange(len(data))
    X, Y = np.meshgrid(E_range, pdf_range)
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.set_xlabel(r'$\mathregular{Log_{10}(E_\nu/GeV)}$')
    ax.set_ylabel('Eigenvector PDF set index')
    ax.set_zlabel(log_title + z_label + r'$\mathregular{(E_\nu,\, i)}$'
                  + z_label_end)
    for index in range(len(plot_data)):
        ax.plot_wireframe(X, Y, plot_data[index], color=colors[index],
                          label=plot_labels[index])
    ax.legend(loc=4)

    if not plot_uncertainties:
        ax.set_title(log_title + title_label +
                     ' DIS cross sections')
    elif plot_uncertainties and uncert_plot_style == 'relative':
        ax.set_title(log_title + title_label +
                     ' DIS cross sections with ' + uncert_plot_method + ' '
                     + uncert_plot_style + ' errors')
    elif plot_uncertainties and uncert_plot_style != 'relative':
        ax.set_title(log_title + title_label +
                     ' DIS cross section ' + uncert_plot_method + ' '
                     + uncert_plot_style + ' errors')
    plt.show()
    return


LO_cross_section('cc', plot_inline=False, log_flag=True,
                 plot_uncertainties=True, uncert_plot_method='asymmetric',
                 uncert_plot_style='relative')


In [4]:
def plot_central_uncertainty(int_type, plot_inline=True, log_flag=False,
                             uncert_plot_method='asymmetric',
                             uncert_plot_style='relative'):
    """
################################################################################
    This function calculates and plots the central cross sections with the
    associated uncertainties for full cross section data (not inelasticity
    differential data) as a function of energy. The variables are defined as
    per 'plot_3d_cross_sections', without the 'plot_uncertainties' flag.
################################################################################
    """

    _plot_check(uncert_plot_method, uncert_plot_style, plot_inline, log_flag,
                True)
    if int_type not in ['cc', 'ccA', 'nc', 'ncA']:
        raise ValueError("This is not a valid int_type - please select "
                         "either 'cc', 'ccA', 'nc', or 'ncA'.")
    if log_flag and uncert_plot_style == 'fractional':
        raise ValueError("This is not a valid combination of log_flag "
                         "and uncert_plot_style. If the desired type of "
                         "plot is fractional, please select log_flag to be "
                         "False.")
    data = read_data(int_type)
    central_pdf = data[0]
    E_range = np.linspace(2, 12, num=len(data[0]))
    plot_data = []
    plot_labels = []

    if plot_inline:
        %matplotlib inline
    else:
        %matplotlib qt

    if 'cc' in int_type:
        central_label = r'$\sigma^{CC}_0$'
    elif 'nc' in int_type:
        central_label = r'$\sigma^{NC}_0$'

    plt.figure()
    plt.xlabel(r'$\mathregular{Log_{10}(E_\nu/GeV)}$')

    y_label, title_label = _labels(int_type, log_flag, uncert_plot_style)

    if uncert_plot_style in ['relative', 'fractional']:
        if log_flag:
            plt.plot(E_range, np.log10(central_pdf), color='black',
                     label=central_label)
        else:
            plt.plot(E_range, central_pdf, color='black', label=central_label)
    if log_flag and uncert_plot_style in ['absolute', 'fractional']:
        print("Warning: The full range of values may not display as many "
              "of the cross section uncertainties from certain individual "
              "PDFs are null, thus by setting log_flag to true this raises "
              "library runtime and divergence errors.")
    if uncert_plot_style == 'fractional':
        frac_flag = True
    else:
        frac_flag = False

    if uncert_plot_method == 'averaged':
        print("Warning: As the uncertainty is bounded in the negative "
              "direction by 100%, using the averaged uncertianties "
              "may lead to negative plots.")
        delta = _cs_uncertainty(data, 'a', _fraction_flag=frac_flag,
                                _sum_flag=True)
        plot_labels, plot_data = _add_plot(central_pdf + delta, plot_data,
                                           log_flag, r'$\Delta\sigma_+$',
                                           plot_labels)
        plot_labels, plot_data = _add_plot(central_pdf - delta, plot_data,
                                           log_flag, r'$\Delta\sigma_-$',
                                           plot_labels)
    elif uncert_plot_method == 'asymmetric':
        deltap = _cs_uncertainty(data, 'p', _fraction_flag=frac_flag,
                                 _sum_flag=True)
        deltam = _cs_uncertainty(data, 'm', _fraction_flag=frac_flag,
                                 _sum_flag=True)
        plot_labels, plot_data = _add_plot(central_pdf + deltap, plot_data,
                                           log_flag, r'$\Delta\sigma_+$',
                                           plot_labels)
        plot_labels, plot_data = _add_plot(central_pdf - deltam, plot_data,
                                           log_flag, r'$\Delta\sigma_-$',
                                           plot_labels)
    elif uncert_plot_method == 'positive':
        delta = _cs_uncertainty(data, 'p', _fraction_flag=frac_flag,
                                _sum_flag=True)
        plot_labels, plot_data = _add_plot(central_pdf + delta, plot_data,
                                           log_flag, r'$\Delta\sigma_+$',
                                           plot_labels)
    elif uncert_plot_method == 'negative':
        delta = _cs_uncertainty(data, 'm', _fraction_flag=frac_flag,
                                _sum_flag=True)
        plot_labels, plot_data = _add_plot(central_pdf - delta, plot_data,
                                           log_flag, r'$\Delta\sigma_-$',
                                           plot_labels)

    if log_flag:
        log_title = r'$\mathregular{Log_{10}}$'
        central_pdf = np.log10(central_pdf)
    else:
        log_title = ''
        title_label = title_label.capitalize()

    plt.ylabel(log_title + y_label + r'$(E_\nu)/cm^{-2}$')
    plt.title(log_title + " " + title_label + ' DIS cross section against '
              'neutrino energy with ' + uncert_plot_style + ' '
              + uncert_plot_method + ' errors')

    colors = ['blue', 'red']
    for index in range(len(plot_data)):
        if uncert_plot_method == 'positive':
            plt.fill_between(E_range, plot_data[index], central_pdf,
                             color='blue', label=plot_labels[index],
                             alpha=1.)
        elif uncert_plot_method == 'negative':
            plt.fill_between(E_range, plot_data[index], central_pdf,
                             color='red', label=plot_labels[index],
                             alpha=1.)
        else:
            plt.fill_between(E_range, plot_data[index], central_pdf,
                             color=colors[index], label=plot_labels[index],
                             alpha=1.)
    plt.legend(loc=2)
    plt.show()
    return


plot_central_uncertainty('cc', plot_inline=False, log_flag=False,
                         uncert_plot_method='asymmetric',
                         uncert_plot_style='fractional')


In [6]:
def plot_comparison(int_type, log_flag=False, plot_inline=True, l_lim=4,
                    u_lim=12, step=0.2, _ret_data=False):
    """
################################################################################
    This function plots the cross sections of the MSTW2008 PDFs presented in
    equation 7 of 1102.0691, and the cross sections resulting from the analysis
    of the MMHT2014 PDFs. The constants used in the analysis of the central
    cross sections and associated uncertainties of the MSTW2008 PDFs are
    presented in tables 3 and 4 of this paper. The variables are:
        1. int_type - this string defines the type of interaction that is
                      being plotted. This should be a choice of the permitted
                      values of the function _read_data aisde from the
                      inelasticity differential data.
        2. log_flag - this flag determines whether the plots are presented in
                      logarithmic form or not.
        3. plot_inline - this flag determines whether the plot should be in a
                         new window (False) or in the output box (True).
        4. l_lim - this float determines the starting log10 of the neutrino
                   energy for the plot
        5. u_lim - this float determines the finishing log10 of the neutrino
                   energy for the plot
        6. step - this float indicates the logE step size. This has one decimal
                  place.
        7. _ret_data - this internal flag determines whether the function
                       returns the data associated with the plots or whether
                       the plots are made.
################################################################################
    """
    int_type_list = ['cc', 'ccA', 'nc', 'ncA']
    if int_type not in int_type_list:
        raise ValueError("This is not a valid int_type - please select either "
                         "'cc', 'ccA', 'nc', or 'ncA'.")
    if type(log_flag) != bool:
        raise ValueError("This is not a valid log_flag - please ensure that "
                         "this is a boolean.")
    if (10. * step) % 1 != 0.0:
        raise ValueError("This is not a valid step size - please ensure that "
                         "the step size is a float with one decimal point "
                         "only.")
    if (10. * l_lim) % (10 * step) != 0.0:
        raise ValueError("This is not a valid l_lim - please ensure that this "
                         "is a multiple of the step.")
    if (10. * u_lim) % (10 * step) != 0.0:
        raise ValueError("This is not a valid u_lim - please ensure that this "
                         "is a multiple of the step.")

    if plot_inline:
        %matplotlib inline
    else:
        %matplotlib qt

    data = read_data(int_type)
    step = 10/(len(data) - 1)
    l_index = int((l_lim - 2)/step)
    u_index = len(data) - int((12 - u_lim)/step)
    x = np.linspace(l_lim, u_lim, int((u_lim - l_lim)/step) + 1)
    y_e = data[0][l_index:u_index]

    int_type_index = int_type_list.index(int_type)
    title_label_list = ['charged current neutrino hadron',
                        'charged current antineutrino hadron',
                        'neutral current neutrino hadron',
                        'neutral current antineutrino hadron']
    y_label_list = [r'$\mathregular{\sigma^{CC}}(\nu_\ell\, H \rightarrow'
                    r'\ell^\mp\, X)/\mathregular{cm^{-2}}$',
                    r'$\mathregular{\sigma^{CC}}(\bar{\nu}_\ell\, H '
                    r'\rightarrow \ell^\mp\, X)/\mathregular{cm^{-2}}$',
                    r'$\mathregular{\sigma^{NC}}(\nu_\ell\, H \rightarrow'
                    r'\nu_\ell\,X)/\mathregular{cm^{-2}}$',
                    r'$\mathregular{\sigma^{NC}}(\bar{\nu}_\ell\,H'
                    r'\rightarrow\bar{\nu}_\ell\,X)/\mathregular{cm^{-2}}$']
    title_label = title_label_list[int_type_index]
    y_label = y_label_list[int_type_index]

    if 'cc' in int_type:
        if 'A' in int_type:
            [c1, c2, c3, c4, c0] = [-15.95, -7.247, 1.569, -17.72, -1.033]
            [c1u, c2u, c3u, c4u, c0u] = [144.5, -77.44, 11.90, -142.8, -2.945]
            [c1l, c2l, c3l, c4l, c0l] = [12.48, 33.52, -8.191, -216.1, -13.08]
        else:
            [c1, c2, c3, c4, c0] = [-17.31, -6.406, 1.431, -17.91, -1.826]
            [c1u, c2u, c3u, c4u, c0u] = [33.47, -33.02, 6.026, -49.41, -1.456]
            [c1l, c2l, c3l, c4l, c0l] = [13.86, 39.84, -9.205, -253.1, -15.35]
    elif 'nc' in int_type:
        if 'A' in int_type:
            [c1, c2, c3, c4, c0] = [-15.95, -7.296, 1.569, -18.30, -1.033]
            [c1u, c2u, c3u, c4u, c0u] = [143.2, -76.70, 11.75, -142.8, -2.945]
            [c1l, c2l, c3l, c4l, c0l] = [15.17, 31.19, -7.757, -216.1, -13.08]
        else:
            [c1, c2, c3, c4, c0] = [-17.31, -6.448, 1.431, -18.61, -1.826]
            [c1u, c2u, c3u, c4u, c0u] = [32.23, -32.32, 5.881, -49.41, -1.456]
            [c1l, c2l, c3l, c4l, c0l] = [16.16, 37.71, -8.801, -253.1, -15.35]

    y = c1 + c2 * np.log(x - c0) + c3 * (np.log(x - c0))**2 +\
        (c4/(np.log(x - c0)))
    yu = c1u + c2u * np.log(x - c0u) + c3u * (np.log(x - c0u))**2 +\
        (c4u/(np.log(x - c0u)))
    yl = c1l + c2l * np.log(x - c0l) + c3l * (np.log(x - c0l))**2 +\
        (c4l/(np.log(x - c0l)))

    deltap = _cs_uncertainty(data, 'p', _sum_flag=True)
    deltam = _cs_uncertainty(data, 'm', _sum_flag=True)
    y_eu = y_e + deltap[l_index:u_index]
    y_el = y_e - deltam[l_index:u_index]
    if log_flag:
        y_e = np.log10(y_e)
        y_eu = np.log10(y_eu)
        y_el = np.log10(y_el)
        log_title = r'$\mathregular{Log_{10}}$'
    else:
        y = 10 ** y
        yu = 10 ** yu
        yl = 10 ** yl
        log_title = ''
        title_label.capitalize()

    if _ret_data:
        return([x, y, yu, yl, y_e, y_eu, y_el])
    else:
        plt.plot(x, y, label='MSTW2008 prediction', color='b')
        plt.fill_between(x, yu, yl, label='MSTW2008 uncertainty',
                         alpha=0.4, color='b')
        plt.plot(x, y_e, label='MMHT2014 calculation', color='k')
        plt.fill_between(x, y_eu, y_el, label='MMHT2014 uncertainty',
                         alpha=0.4, color='k')
        plt.title('Comparing the MMHT2014 and  MSTW2008 ' + title_label +
                  ' DIS cross sections')
        plt.xlabel(r'$\mathregular{Log_{10}(E_\nu/GeV)}$')
        plt.ylabel(log_title + y_label + r'$\mathregular{(E_\nu)}$')
        plt.legend(loc=2)
        plt.show()
        return


plot_comparison('cc', log_flag=True, plot_inline=False, l_lim=4, u_lim=12,
                _ret_data=False)


In [9]:
def plot_all_comparison(nu_type, log_flag=False, plot_inline=True, l_lim=4,
                        u_lim=12, step=0.2):
    """
################################################################################
    This function plots the neutral and charged current, and total cross
    sections in the parameterised form of the MSTW2008 PDFs presented in
    equation 7 of 1102.0691 and the cross sections resulting from the analysis
    of the MMHT2014 PDFs. The constants used in the analysis of the central
    cross section and associated uncertainties is presented in tables 3 and 4
    of this paper. The variables are:
        1. nu_type - this string defines the type of neutrino interaction
                     that is being plotted.
            * nu - this plots the neutrino hadron cross sections.
            * nubar - this plots the antineutrino hadron cross sections.
        2. plot_inline - this flag determines whether the plot should be
                         presented the console (True) or in a new window
                         (False).
        3. log_flag - this flag determines whether the plots are presented in
                      logarithmic form (True) or not (False).
        3. l_lim - this float determines the starting log10 of the neutrino
                   energy for the plot
        4. u_lim - this float determines the finishing log10 of the neutrino
                   energy for the plot
        5. step - this float indicates the logE step size. This has one decimal
                  place.
################################################################################
    """
    if nu_type not in ['nu', 'nubar']:
        raise ValueError("This is not a valid type of nu_type - please select "
                         "either 'nu' or 'nubar'.")
    full_data = []
    if nu_type == 'nu':
        title_label = 'neutrino hadron'
        for int_type in ['cc', 'nc']:
            full_data.append(plot_comparison(int_type, log_flag=log_flag,
                                             plot_inline=plot_inline,
                                             l_lim=l_lim, u_lim=u_lim,
                                             _ret_data=True))
    elif nu_type == 'nubar':
        title_label = 'antineutrino hadron'
        for int_type in ['ccA', 'ncA']:
            full_data.append(plot_comparison(int_type, log_flag=log_flag,
                                             plot_inline=plot_inline,
                                             l_lim=l_lim, u_lim=u_lim,
                                             _ret_data=True))
    E_range = full_data[0][0]
    plt.xlabel(r'$\mathregular{Log_{10}(E_\nu /GeV)}$')
    if log_flag:
        plt.ylabel(r'$\mathregular{Log_{10}(\sigma/cm^{-2}})$')
    else:
        plt.ylabel(r'$\mathregular{\sigma/cm^{-2}}$')
    if log_flag:
        total_sigma = np.log10(10**full_data[0][1] + 10**full_data[1][1])
        total_sigma_e = np.log10(10**full_data[0][4] + 10**full_data[1][4])
    else:
        total_sigma = full_data[0][1] + full_data[1][1]
        total_sigma_e = full_data[0][4] + full_data[1][4]

    plt.plot(E_range, total_sigma,
             label=r'$\mathregular{\sigma^{CC}_{total}}$ MSTW2008', color='m')
    plt.plot(E_range, total_sigma_e,
             label=r'$\mathregular{\sigma^{CC}_{total}}$ MMHT2014', color='y')

    plt.plot(E_range, full_data[0][1], '-.',
             label=r'$\sigma^{CC}$ MSTW2008', color='b')
    plt.fill_between(E_range, full_data[0][2], full_data[0][3], alpha=0.4,
                     color='b', label=r'$\sigma^{CC}$ MSTW2008 uncertainty')

    plt.plot(E_range, full_data[0][4], '-.', label=r'$\sigma^{CC}$ MMHT2014',
             color='k')
    plt.fill_between(E_range, full_data[0][5], full_data[0][6], alpha=0.4,
                     color='k', label=r'$\sigma^{CC}$ MMHT2014 uncertainty')

    plt.plot(E_range, full_data[1][1], '--', label=r'$\sigma^{NC}$ MSTW2008',
             color='g')
    plt.fill_between(E_range, full_data[1][2], full_data[1][3], alpha=0.4,
                     color='g', label=r'$\sigma^{NC}$ MSTW2008 uncertainty')

    plt.plot(E_range, full_data[1][4], '--', label=r'$\sigma^{NC}$ MMHT2014',
             color='r')
    plt.fill_between(E_range, full_data[1][5], full_data[1][6], alpha=0.4,
                     color='r', label=r'$\sigma^{NC}$ MMHT2014 uncertainty')

    plt.title('Comparing the ' + title_label + ' DIS cross sections '
              'for MSTW2008 and MMHT2014 PDFs')
    plt.legend(loc=2)
    plt.show()
    return


plot_all_comparison('nu', log_flag=True, plot_inline=False, l_lim=8)


In [11]:
def plot_3d_cross_section_y(int_type, plot_inline=True, log_flag=True,
                            plot_uncertainties=False,
                            uncert_plot_method='asymmetric',
                            uncert_plot_style='relative'):
    """
################################################################################
    This function plots the inelasticity differential cross section data (with
    the associated uncertainties) in three dimensions as a function of the
    energy and the inelasticity, as opposed to eigenvector PDF
    distributions in 'plot_3d_cross_sections'. The variables are defined as per
    'plot_3d_cross_sections', with the difference entailing the choice of
    int_type - in this function the int_type has to be a rapidity distributed
    set.
################################################################################
    """
    _plot_check(uncert_plot_method, uncert_plot_style, plot_inline,
                log_flag, plot_uncertainties)
    if int_type not in ['ncY', 'ccY', 'ncAY', 'ccAY']:
        raise ValueError("This is not a valid type of interaction for this "
                         "function - please select either 'ncY', 'ccY', "
                         "'ncAY', or 'ccAY'. If you would like to see the "
                         "three dimensional distribution for fully integrated "
                         "data, refer to the 'plot_3d_cross_section' "
                         "function.")

    if plot_inline:
        %matplotlib inline
    else:
        %matplotlib qt

    data = read_data(int_type)
    z_label, title_label = _labels(int_type, log_flag, uncert_plot_style)

    plot_items = _construct_plots(int_type, log_flag, plot_uncertainties,
                                  uncert_plot_method, uncert_plot_style,
                                  True, _cs_y_uncertainty)
    plot_data, plot_labels, = plot_items[0], plot_items[1]
    colors, log_title = plot_items[2], plot_items[3]
    z_label_end = plot_items[4]

    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.set_ylabel(r'$\mathregular{Log_{10}(E_\nu/GeV)}$')
    ax.set_xlabel(r'$\mathregular{Log_{10}(Inelasticity)}\ (y)$')
    ax.set_zlabel(log_title + z_label + z_label_end)
    y_range = np.linspace(-5, 0, num=len(data[0][0]))
    energy_range = np.linspace(7, 12, num=len(data[0]))
    X, Y = np.meshgrid(y_range, energy_range)

    for index in range(len(plot_data)):
        ax.plot_wireframe(X, Y, plot_data[index], color=colors[index],
                          label=plot_labels[index])
    ax.legend(loc=4)

    if plot_uncertainties:
        ax.set_title(log_title + title_label +
                     ' DIS inelasticity differential cross sections with ' +
                     uncert_plot_method + ' ' + uncert_plot_style + ' errors')
    else:
        ax.set_title(log_title + title_label +
                     ' DIS inelasticity differential cross sections.')
    plt.show()
    return


plot_3d_cross_section_y('ccY', plot_inline=False, log_flag=True,
                        plot_uncertainties=True,
                        uncert_plot_method='asymmetric',
                        uncert_plot_style='relative')


In [13]:
def plot_2d_inelasticity(int_type, plot_inline=True, section='high', step=0.1,
                         energy=12., plot_comp=False, plot_uncertainties=True):
    """
################################################################################
    This function plots the relative inelasticity distribution as a function
    of the inelasticity y at a fixed energy. The variables are:
        1. int_type - this string defines the type of interaction that is
                      being plotted. This should be a choice of the permitted
                      values of the function _read_data with 'Y' on the end.
        2. plot_inline - this flag determines whether the plot should be in a
                         new window (False) or in the output box (True).
        3. section - this string determines which inelasticity range is
                     plotted.
            * high - this plots the inelasticity in the range [10^-5, 10^-3].
            * low - this plots the inelasticity in the range [10^-3, 1.]
        5. step - this float indicates the logE step size. This has one decimal
                  place.
        6. energy - this float indicates the energy at which the inelasticity
                    is plotted. This has to fall in the range [7, 12].
################################################################################
    """
    int_type_list = ['ccY', 'ccAY', 'ncY', 'ncAY']
    if int_type not in int_type_list:
        raise ValueError("This is not a valid int_type - please select either "
                         "'ccY', 'ccAY', 'ncY', or 'ncAY'.")
    if section not in ['high', 'low']:
        raise ValueError("This is not a valid section title of y values - "
                         "please select either 'long' for logE in [-3,0] or '"
                         "short' for logE in [-5,-3].")
    if type(plot_inline) != bool:
        raise ValueError("This is not a valid type of plot_inline - please "
                         "tensure hat this quantity is a boolean.")
    if (10. * step) % 1 != 0.0:
        raise ValueError("This is not a valid step size - please ensure that "
                         "the step size is a float with one decimal point "
                         "only.")
    if energy > 12. or energy < 7.:
        raise ValueError("This is not a valid energy - please ensure that the "
                         "passed quantity is within the range [7, 12].")
    if plot_inline:
        %matplotlib inline
    else:
        %matplotlib qt

    energy_index = int((energy - 7)/step)
    ref = read_data(int_type[:2])[0][energy_index]

    y_label, title_label = _labels(int_type, False, 'asymmetric')
    range_title_list = [r'for y in $[10^{-5}, 10^{-3}]$',
                        r'for y in $[10^{-3}, 1]$']

    if section == 'low':
        data = read_data(int_type)[0][energy_index][:int((2/step)) + 1]/ref
        range_title = range_title_list[0]
        y_range = np.linspace(-5, -3, len(data))

        if plot_comp:
            A0, A1, A2, A3, B0, B1 = 0, 0.0941, 4.72, 0.456, 2.55, -0.0949
            C1 = A0 - A1 * (np.exp(-((energy-A2)/A3)))
            C2 = B0 + B1 * energy
            C0 = data[-1]*ref*((10**y_range[-1] - C1)**(1/C2))
            data_pred = (C0/((10**y_range - C1)**(1/C2)))/ref
            plt.plot(y_range, data_pred, label="MSTW 2008 Predictions",
                     color='r')

    elif section == 'high':
        data = read_data(int_type)[0][energy_index][int((2/step)) + 1:]/ref
        range_title = range_title_list[1]
        y_range = np.linspace(-3, 0, len(data))

        if plot_comp:
            if int_type[:2] == 'cc':
                if int_type[2] == 'A':
                    A0, A1, A2 = -0.0026, 0.085, 4.1
                else:
                    A0, A1, A2 = -0.008, 0.26, 3
            elif int_type[:2] == 'nc':
                A0, A1, A2 = -0.005, 0.23, 3
            A3 = 1.7
            C1 = A0 - A1 * (np.exp(-((energy-A2)/A3)))
            C0 = data[0]*ref*(10**y_range[0] - C1)
            data_pred = (C0/((10**y_range) - C1))/ref
            plt.plot(y_range, data_pred, label="MSTW 2008 Predictions",
                     color='r')

    plt.plot(y_range, data, label="MMHT 2014 Calculations", color='b')
    if plot_uncertainties:
        uncert_data = read_data(int_type)[:, energy_index]/ref
        if section == 'low':
            deltap = _cs_uncertainty(uncert_data, 'p',
                                     _sum_flag=True)[:int((2/step)) + 1]
            deltam = _cs_uncertainty(uncert_data, 'm',
                                     _sum_flag=True)[:int((2/step)) + 1]
        else:
            deltap = _cs_uncertainty(uncert_data, 'p',
                                     _sum_flag=True)[int((2/step)) + 1:]
            deltam = _cs_uncertainty(uncert_data, 'm',
                                     _sum_flag=True)[int((2/step)) + 1:]
        plt.fill_between(y_range, data+deltap, data-deltam,
                         label='MMHT 2014 Uncertainty', color='b', alpha=0.3)
    plt.title(title_label + ' inelasticity differential cross section '
              + range_title + r' at $\mathregular{Log}_{10}(E_\nu)$ = '
              + str(energy) + ' GeV')
    plt.xlabel(r'$\mathregular{Log_{10}(Inelasticity)}\ y$')
    plt.ylabel(r'$\frac{1}{\sigma^{tot}}$' + y_label)
    plt.legend()
    plt.show()
    return


plot_2d_inelasticity('ccY', plot_inline=False, section='high',
                     energy=12, plot_comp=True)


In [15]:
def charmed_plot(plot_inline=True, plot_uncertainties=True,
                 uncert_plot_method='asymmetric',
                 uncert_plot_style='relative'):
    """
################################################################################
    This function produces the plots for the dimuon graphs. The variables are
    defined as per LO_cross_section without the int_type variable, as this plot
    reqires both 'cc' and 'ccA' data.
################################################################################
    """

    _plot_check(uncert_plot_method, uncert_plot_style, plot_inline,
                True, plot_uncertainties)
    num = (read_data('charm')+read_data('charmA'))
    dnum = (read_data('ccS')+read_data('ccAS'))
    data = num/dnum
    energy_range = np.linspace(2, 4, len(data[0]))
    if uncert_plot_style == 'relative':
        plt.plot(energy_range, data[0],
                 label="Central fraction", color='k')
    elif uncert_plot_style == 'fractional':
        plt.plot(energy_range, np.ones(len(data[0])),
                 label="Central fraction", color='k')
    if plot_uncertainties and uncert_plot_style != 'fractional':
        if uncert_plot_method == 'averaged':
            delta = _c_frac_uncertainty(data, 'a')
            labelp = r'$\Delta\frac{\sigma^{CC}_{charm}}{\sigma^{CC}}^+$'
            labelm = r'$\Delta\frac{\sigma^{CC}_{charm}}{\sigma^{CC}}^-$'
            plt.fill_between(energy_range, data[0] + delta,
                             data[0], color='b',
                             alpha=0.6, label=labelp)
            plt.fill_between(energy_range, data[0] - delta,
                             data[0], color='r',
                             alpha=0.6, label=labelm)
        if uncert_plot_method == 'asymmetric':
            deltap = _cs_uncertainty(data, 'p', _sum_flag=True)
            labelp = r'$\Delta\frac{\sigma^{CC}_{charm}}{\sigma^{CC}}^+$'
            deltam = _cs_uncertainty(data, 'm', _sum_flag=True)
            labelm = r'$\Delta\frac{\sigma^{CC}_{charm}}{\sigma^{CC}}^-$'
            plt.fill_between(energy_range, data[0] + deltap,
                             data[0], color='b',
                             alpha=0.6, label=labelp)
            plt.fill_between(energy_range, data[0] - deltam,
                             data[0], color='r',
                             alpha=0.6, label=labelm)
        if uncert_plot_method == 'positive':
            delta = _c_frac_uncertainty(data, 'p')
            labeld = r'$\Delta\frac{\sigma^{CC}_{charm}}{\sigma^{CC}}^+$'
            plt.fill_between(energy_range, data[0],
                             data[0] + delta, color='b', alpha=0.6,
                             label=labeld)
        if uncert_plot_method == 'negative':
            delta = _c_frac_uncertainty(data, 'm')
            labeld = r'$\Delta\frac{\sigma^{CC}_{charm}}{\sigma^{CC}}^-$'
            plt.fill_between(energy_range, data[0],
                             data[0] - delta, color='r', alpha=0.6,
                             label=labeld)
    elif plot_uncertainties:
        if uncert_plot_method == 'averaged':
            delta = _c_frac_uncertainty(data, 'a')
            labelp = r'$\Delta\frac{\sigma^{CC}_{charm}}{\sigma^{CC}}^+$'
            labelm = r'$\Delta\frac{\sigma^{CC}_{charm}}{\sigma^{CC}}^-$'
            plt.fill_between(energy_range, (data[0]+delta)/data[0],
                             np.ones(len(data[0])), color='b', alpha=0.6,
                             label=labelp)
            plt.fill_between(energy_range, (data[0]-delta)/data[0],
                             np.ones(len(data[0])), color='r', alpha=0.6,
                             label=labelm)
        if uncert_plot_method == 'asymmetric':
            deltap = _c_frac_uncertainty(data, 'p')
            deltam = _c_frac_uncertainty(data, 'm')
            labelp = r'$\Delta\frac{\sigma^{CC}_{charm}}{\sigma^{CC}}^+$'
            labelm = r'$\Delta\frac{\sigma^{CC}_{charm}}{\sigma^{CC}}^-$'
            plt.fill_between(energy_range, (data[0]+deltap)/data[0],
                             np.ones(len(data[0])), color='b', alpha=0.6,
                             label=labelp)
            plt.fill_between(energy_range, (data[0]-deltam)/data[0],
                             np.ones(len(data[0])), color='r', alpha=0.6,
                             label=labelm)
        if uncert_plot_method == 'positive':
            delta = _c_frac_uncertainty(data, 'p')
            labelp = r'$\Delta\frac{\sigma^{CC}_{charm}}{\sigma^{CC}}^+$'
            plt.fill_between(energy_range, (data[0]+delta)/data[0],
                             np.ones(len(data[0])), color='b', alpha=0.6,
                             label=labelp)
        if uncert_plot_method == 'negative':
            delta = _c_frac_uncertainty(data, 'm')
            labelm = r'$\Delta\frac{\sigma^{CC}_{charm}}{\sigma^{CC}}^-$'
            plt.fill_between(energy_range, (data[0]-delta)/data[0],
                             np.ones(len(data[0])), color='r', alpha=0.6,
                             label=labelm)

    plt.xlabel(r'$\mathregular{Log_{10}(E_\nu/GeV)}$')
    plt.ylabel(r'$\left(\mathregular{\sigma^{CC}_{charm}}(\bar{\nu}_\mu N'
               r'\rightarrow \mu^\pm c X)+\mathregular{\sigma^{CC}_{charm}}'
               r'(\nu_\mu N \rightarrow \mu^\mp c X)\right)/'
               r'\mathregular{\sigma^{CC}}$')
    plt.title('Fraction of dimuon production cross sections against '
              'neutrino energy')
    plt.legend(loc=4)
    plt.show()


charmed_plot(uncert_plot_style='relative', uncert_plot_method='asymmetric')


In [16]:
def plot_K(K_factor, remove_K3_outliers=False, n_outliers=1):
    """
################################################################################
    This function plots the K factors in 3d as a function of the parton
    momentum fraction x and the virtuality Q^2. The variables are:
        1. k_factor - This determines which K-factor is plotted.
            * K2 - This is the ratio of the F_2 structure functions as NLO:LO
            * K3 - This is the ratio of the F_3 structure functions as NLO:LO
            * KL - This is the ratio of the F_L structure functions as NLO:LO.
                   The discrepancy between this method and the previous relates
                   to the theory of KL at LO - here the K factor is computed as
                   a ration between KL at NLO and K2 at LO.
        2. remove_K3_outliers - This flag determines whether the K3 outliers
                                are removed. K_factor has to be 'K3' for this
                                flag to be applied.
        3. n_outliers - This positive int determines the number of outliers
                        removed from the K3 plot.
################################################################################
    """
    if K_factor not in ['K2', 'K3', 'KL']:
        raise ValueError("This is not a valid type of k_factor - please "
                         "select either 'K2', 'K3' or 'KL'.")
    if type(remove_K3_outliers) != bool:
        raise TypeError("This is not a valid type of remove_K3_outliers - "
                        "please ensure that the passed remove_K3_outliers "
                        "is a boolean.")
    if type(n_outliers) != int or n_outliers < 1:
        raise ValueError("This is not a valid type of n_outliers, please "
                         "ensure that this is a positive integer.")
    if remove_K3_outliers and K_factor != 'K3':
        raise ValueError("This is not a valid combination of "
                         "remove_K3_outliers and K_factor - "
                         "remove_K3_outliers only applies for the K3 "
                         "K_factor. Please either set remove_K3_outliers to "
                         "False to plot this data set with the desired number "
                         "of outliers removed, or select K_factor = 'K3' to "
                         "continue using remove_K3_outlier.")
    if K_factor == 'K3' and not remove_K3_outliers and n_outliers != 1:
        raise ValueError("This is not a valid combination of "
                         "remove_K3_outliers and n_outliers - please ensure "
                         "that remove_K3_outliers is True before setting the "
                         "desired number of outliers to be removed.")

    lo = read_data('toyLO')
    nlo = read_data('toyNLO')
    x_data = np.log10(lo[:, 1])
    Q2_data = np.log10(lo[:, 2])
    x, q2 = np.meshgrid(np.linspace(-9, 0, 19), np.linspace(0, 8.5, 18))
    if K_factor == 'K2':
        a1, a2 = 0.91669548084686067, -0.0291359802747228715
        a3, a4 = 0.00250441262910724807, 0.000413866855168086859
        b1, b2 = 0.073460197836531998, -0.0148441479480977412
        b3, c1 = 0.00090909662686127497, 0.060950510300603660
        c2, c3 = 0.0063940185101863625, 0.000136409968463251112
        c4, c5 = -0.0142482417209574975, -0.00182117732782461936
        c6, c7 = -0.000056627023799601716, 0.00092561268369528562
        c8, c9 = 0.000127008992902106137, 4.31114973076896706*10**(-6)
        z_label = r'$\mathregular{K_2(x,\, Q^2)}$'
        data = nlo[:, 3]/lo[:, 3]
    elif K_factor == 'K3':
        a1, a2 = 0.93685841428738469, 0.90108100945053846
        a3, a4 = 0.130148267214913217, 0.00157287350066311651
        b1, b2 = 0.58528415029902340, -0.59461001046137844
        b3, c1 = 0.068763986060213908, 0.425086396849457826
        c2, c3 = 0.177166961431914888, 0.0202185231769084751
        c4, c5 = -0.53611097165897923,  -0.135626568033609870
        c6, c7 = -0.0102167565293350299, 0.061546394969707958
        c8, c9 = 0.0146940606805018030, 0.00104135
        z_label = r'$\mathregular{K_3(x,\, Q^2)}$'
        data = nlo[:, 5]/lo[:, 5]
        for i in range(n_outliers):
            data[np.where(abs(data) == max(abs(data)))[0][0]] = 0
    elif K_factor == 'KL':
        a1, a2 = 0.126975812344098490, -0.149697545958257405
        a3, a4 = -0.0379705487394227540, -0.00256084672955436150
        b1, b2 = -0.121084789582025436, 0.0273871186035903870
        b3, c1 = -0.00179866852119628995, 0.0137662438218560166
        c2, c3 = 0.0142780510726009759, 0.00123220029717474596
        c4, c5 = 0.00285552956739174058, -0.00214681055835942860
        c6, c7 = -0.000218792886420921078, -0.000341580983402590611
        c8, c9 = 0.000112996270241938136, 0.0000128379589669445821
        z_label = r'$\mathregular{K_L(x,\, Q^2)}$'
        data = nlo[:, 4]/lo[:, 3]
    fitted_data = (a1 + (a2*x) + (a3*x**2) + (a4*x**3) + (b1*q2) + (b2*q2**2)
                   + (b3*q2**3) + (c1*x*q2) + (c2*(x**2)*q2) + (c3 * (x**3)*q2)
                   + (c4*x*(q2**2)) + (c5*(x**2)*(q2**2))
                   + (c6*(x**3)*(q2**2)) + (c7*x*(q2**3))
                   + (c8*(x**2)*(q2**3)) + (c9*(x**3)*(q2**3)))
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.set_xlabel(r'$\mathregular{Log_{10}(x)}$')
    ax.set_ylabel(r'$\mathregular{Log_{10}(Q^2/GeV})$')
    ax.set_zlabel(z_label)
    ax.plot_surface(x, q2, fitted_data, alpha=0.6)
    ax.scatter(x_data, Q2_data, data + 10**(-2), label='Data', color='r')
    ax.set_title('A plot of ' + z_label
                 + r' data and the corresponding fitted function')
    ax.legend()
    return


plot_K('K2', remove_K3_outliers=False, n_outliers=10)


In [131]:
def plot_order_comparison():
    """
################################################################################
    This function plots the fractional comparison between the LO and NLO data
    sets. The uncertainty_plot_method is fixed to be 'asymmetric' by the nature
    of the script. The spikes in the data here are not physical, they are to do
    with the numerical effects occuring when the number of recursions and the
    working precision is capped to 5.
################################################################################
    """
    dataLO = read_data('ccLO')
    dpLO = _cs_uncertainty(dataLO, method='p', _sum_flag=True)
    dmLO = _cs_uncertainty(dataLO, method='m', _sum_flag=True)

    dataNLO = read_data('ccNLO')
    dpNLO = _cs_uncertainty(dataNLO, method='p', _sum_flag=True)
    dmNLO = _cs_uncertainty(dataNLO, method='m', _sum_flag=True)

    E_rangeLO = np.linspace(2, 12, num=len(dataLO[0]))
    E_rangeNLO = np.linspace(2, 12, num=len(dataNLO[0]))

    dfp = _cs_uncertainty(dataNLO/dataLO, method='p', _sum_flag=True)
    dfm = _cs_uncertainty(dataNLO/dataLO, method='m', _sum_flag=True)

    plt.plot(E_range, dataNLO[0]/dataLO[0], color='black',
             label=r'$\frac{\sigma^{NLO}}{\sigma^{LO}}$')
    plt.fill_between(E_range, dataNLO[0]/dataLO[0], (dataNLO[0]/dataLO[0])+dfp,
                     color='blue', alpha=0.6,
                     label=r'$\Delta\frac{\sigma^{NLO}}{\sigma^{LO}}^+$')
    plt.fill_between(E_range, dataNLO[0]/dataLO[0], (dataNLO[0]/dataLO[0])-dfm,
                     color='red', alpha=0.6,
                     label=r'$\Delta\frac{\sigma^{NLO}}{\sigma^{LO}}^-$')
    plt.xlabel(r'$\mathregular{Log_{10}(E_\nu/GeV)}$')
    plt.ylabel(r'$\mathregular{\sigma^{NLO}/\sigma^{LO}}$')
    plt.title('A comparison of the LO to the NLO cross sections')
    plt.legend(loc=2)
    plt.show()
    return


plot_order_comparison()
